In [1]:
import numpy as np
import pandas as pd

In [2]:
#datasets imported from R code
train = pd.read_csv("data/ml_data.csv")
test = pd.read_csv("data/ml_testdata.csv")

train50 = pd.read_csv("data/train50.csv")#training data with d=50
test50 = pd.read_csv("data/test50.csv")#testing data with d=50
test

,y,x1,x2,x3,x4,x5,X1,X2,X3,X4,X5
0,-61.291725,4.447555,-3.361047,-2.274540,1.852450,1,1.222541,-0.718750,-0.092678,-1.832146,0.541431
1,-37.527565,3.725128,-1.259678,1.205772,0.548077,1,1.466484,-2.202273,0.495814,-2.325496,2.913829
2,-78.563960,5.366005,-0.579791,4.206422,2.016993,1,0.930874,-1.194001,0.843049,-0.352840,-1.076339
3,-5.450604,1.500686,-1.401609,-1.302531,0.358072,1,0.466832,-0.260097,0.912008,-0.510121,-2.085178
4,-7.016755,1.817803,-1.182545,-0.547288,0.668232,1,0.159126,0.835107,0.977348,0.370391,-0.149051
...,...,...,...,...,...,...,...,...,...,...,...
995,-30.633786,3.528683,-0.298072,2.932539,2.896558,1,2.255478,-0.746831,-0.247752,-0.632649,-0.959097
996,-32.880433,3.343772,-2.263303,-1.182834,0.069328,1,0.352842,0.762762,-1.033225,0.476829,0.954998
997,-16.332526,2.465531,-2.017796,-1.570061,0.000317,0,0.294179,-2.211614,-0.276436,0.065659,0.254701
998,-14.898616,2.245702,-2.903066,-3.560431,0.815529,0,0.242152,-0.862668,0.280049,-1.327336,0.799153


In [3]:
#decision tree truncating by depth
    #y_average: predicted y vector that will be returned when the algorithm terminates
    #depth: variable that indicates the current depth the recursive algorithm it is on
    #maxdepth: maximum depth the tree reaches before returning the predicted y-value

def decisiontreedepth(data, y_average, depth, maxdepth):
    
    #stopping conditions, assigns predicted y-values
    if((depth >= maxdepth) | (len(data) <= 1)):
        for d in data.index:
            y_average[d] = np.mean(data['y'])      
            
        return y_average
    
    
    maxcor = 0 #maximum correlation to y
    xi = ''# x variable with maximum correlation to y
    
    
    #get highest correlated xi value for decision tree
    for i in data:        
        r = np.corrcoef(data['y'], data[i])
            
        if((abs(r[0,1]) > maxcor) & (i != 'y')):
            maxcor = abs(r[0,1])          
            xi = i
    
    #sort data by xi
    sortedx = data.sort_values(by=xi)
    
    minerror = 0
    minindex = 0
    
    #find split of data that gives the smallest total error
    for k in range(len(data)-2):        
        leftdata = sortedx.iloc[:(k+1), :]
        rightdata = sortedx.iloc[(k+1):, :]
            
        lefterror = sum((leftdata['y'] - (np.mean(leftdata['y'])))**2)/len(leftdata)
        righterror = sum((rightdata['y'] - (np.mean(rightdata['y'])))**2)/len(rightdata)
        
        totalerror = ((len(leftdata)/len(data))*lefterror) + ((len(rightdata)/len(data))*righterror)
        
        if(k == 0 or totalerror < minerror):
            minerror = totalerror
            minindex = k + 1
                    
        
    
    if(len(data) == 2):
        minindex = 1
    
    #splits the dataset into two subsets for recursion
    ldata = sortedx.iloc[:minindex, :]
    rdata = sortedx.iloc[minindex:, :]
    
    #recursively constructs decision tree until maximum depth is reached
    y_average = decisiontreedepth(ldata, y_average, depth+1, maxdepth)
    y_average = decisiontreedepth(rdata, y_average, depth+1, maxdepth)
    
    return y_average
    
    

In [14]:
#individual tree testing
ymean = test['y'].copy()
decision_tree = decisiontreedepth(test, ymean, 0, 5)
print(decision_tree)

0     -19.314724
1     -22.941665
2     -57.364206
3     -29.375196
4     -29.375196
         ...    
995   -22.941665
996   -44.239831
997    -6.900236
998   -15.303169
999   -22.941665
Name: y, Length: 1000, dtype: float64


In [4]:
#calculates the errors of decision trees over a range of depth values. These errors will be used for plotting.
y_actual = test['y'].copy()

errors = []
for e in range(1, 9):
    y_pred = test['y'].copy()
    errors.append(sum((decisiontreedepth(test, y_pred, 0, e)-y_actual)**2)/len(test))
print(errors)
#print(sum((decision_tree-y_actual)**2)/len(test))

C:\Users\castl\anaconda3\lib\site-packages\numpy\lib\function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\castl\anaconda3\lib\site-packages\numpy\lib\function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


[112.75843054064276, 38.17382246154596, 14.927364230670149, 7.696119925886805, 4.111621033036673, 2.196420476538805, 1.1041325614453166, 0.48130915699160387]


In [7]:
#decision tree truncating by sample size
    #y_average: predicted y vector that will be returned when the algorithm terminates
    #minsize: minimum sample size the branches can have before returning the predicted y-value

def decisiontreesize(data, y_average, minsize):
    
    #stopping conditions, assigns predicted y-values
    if(len(data) <= minsize):       
        for d in data.index:
            y_average[d] = np.mean(data['y'])
            
        return y_average
    
    
    maxcor = 0 #maximum correlation to y
    xi = '' #x variable with maximum correlation to y
    
    
    #get highest correlated xi value for decision tree
    for i in data:        
        r = np.corrcoef(data['y'], data[i])
            
        if((abs(r[0,1]) > maxcor) & (i != 'y')):
            maxcor = abs(r[0,1])          
            xi = i
    
    #sorts data by xi
    sortedx = data.sort_values(by=xi)
    
    minerror = 0
    minindex = 0
    
    #find split of data that gives the smallest total error
    for k in range(len(data)-2):
        leftdata = sortedx.iloc[:(k+1), :]
        rightdata = sortedx.iloc[(k+1):, :]
            
        lefterror = sum((leftdata['y'] - (np.mean(leftdata['y'])))**2)/len(leftdata)
        righterror = sum((rightdata['y'] - (np.mean(rightdata['y'])))**2)/len(rightdata)
        
        totalerror = ((len(leftdata)/len(data))*lefterror) + ((len(rightdata)/len(data))*righterror)
        
        if(k == 0 or totalerror < minerror):
            minerror = totalerror
            minindex = k + 1
                    
    
    
    if(len(data) == 2):
        minindex = 1
    
    #splits the dataset into two subsets for recursion
    ldata = sortedx.iloc[:minindex, :]
    rdata = sortedx.iloc[minindex:, :]
    
    
    #recursively constructs decision tree until minimum sample size is reached or passed
    y_average = decisiontreesize(ldata, y_average, minsize)
    y_average = decisiontreesize(rdata, y_average, minsize)
    
    return y_average
    
    
    

In [12]:
#individual tree testing
ymean2 = test['y'].copy()
decision_tree2 = decisiontreesize(test, ymean2, 50)
print(decision_tree2)

0     -19.314724
1     -22.078372
2     -58.495591
3     -27.388726
4     -31.692989
         ...    
995   -24.306224
996   -46.823721
997    -7.086996
998   -15.303169
999   -22.078372
Name: y, Length: 1000, dtype: float64


In [10]:
#calculates the errors of decision trees over a range of sample sizes. These errors will be used for plotting.
y_actual2 = test['y'].copy()

sizes = [999, 500, 250, 100, 50, 25, 10, 5]
errors2 = []
for s in sizes:
    y_pred2 = test['y'].copy()
    errors2.append(sum((decisiontreesize(test, y_pred2, s)-y_actual2)**2)/len(test))
print(errors2)
#print(sum((decision_tree2-y_actual2)**2)/len(df))

[115.18155296037939, 77.24071091974238, 28.111645245591912, 18.636768880280474, 5.950090990631368, 3.7131819564608968, 1.0036260343775647, 0.5292267625359737]
